### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

# Simple bot in DeepPavlov

This tutorial describes how to build a simple trainable dialogue system with DeepPavlov framework. It shows one of the easiest ways to create a chatbot. All you need is just a dozen of dialogs from your domain with bot responses annotated for dialogue acts. The tutorial covers the following steps:

0. [Data preparation](#0.-Data-Preparation)
1. [Train bot](#1.-Train-bot)
2. [Interact with bot](#2.-Interact-with-bot)


An example of the final model served as a telegram bot is:

![gobot_simple_example.png](img/gobot_simple_example.png)

In [ ]:
!pip install deeppavlov
!python -m deeppavlov install gobot_dstc2_minimal

## 0. Data Preparation

In this tutorial we will build and train a simple chatbot just from 10 dialogues. 

Reading data:

In [ ]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader


class AssistantDatasetReader(SimpleDSTC2DatasetReader):
    
    url = "http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz"
    
    @staticmethod
    def _data_fname(datatype):
        assert datatype in ('val', 'trn', 'tst'), "wrong datatype name"
        return f"assistant-{datatype}.json"

In [ ]:
data = AssistantDatasetReader().read('assistant_data')

The training/validation/test data is stored in json files (`assistant-trn.json`, `assistant-val.json` and `assistant-tst.json`):

In [ ]:
!ls assistant_data

Let's take a look at the training data.

In [ ]:
!head -n 310 assistant_data/assistant-trn.json

Create data iterator to organize data processing.

In [ ]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed dialogs:

In [ ]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

print("\n-----------------")    
print(f"{len(iterator.get_instances('train')[0])} dialog(s) in train.")
print(f"{len(iterator.get_instances('valid')[0])} dialog(s) in valid.")
print(f"{len(iterator.get_instances('test')[0])} dialog(s) in test.")

## 1. Train bot

A policy module of the bot decides what action should be taken in the current dialogue state.The policy in our bot is implemented as a recurrent neural network (recurrency over user utterances) followed by a dense layer with softmax function on top. The network classifies user input into one of predefined system actions. Examples of possible actions are to say hello, to ask what is the weather or to suggest to drink tea. 

&nbsp;
![gobot_simple_policy.png](img/gobot_simple_policy.png)
&nbsp;

All actions available for the system should be listed in a `assistant-templates.txt` file. Each action should be associated with a string of the corresponding system response.

&nbsp;
![gobot_simple_templates.png](img/gobot_simple_templates.png)
&nbsp;

Templates should be in the format `<act>TAB<template>`, where `<act>` is a dialogue action and `<template>` is the corresponding response.

List of actions for our bot:

In [ ]:
!head -n 10 assistant_data/assistant-templates.txt

In essense, the dialogue policy module solves classification task, where a set of classes is defined in `assistant-templates.txt`. So, to train the dialogue policy network you need action label for each system's turn in training dialogues. Our assistant dataset provides `"act"` dictionary key that contains action associated with current response. Here is an example of training data for the policy network.

In [ ]:
!head -n 31 assistant_data/assistant-trn.json

For our bot we will use ML pipline for task-oriented conversational skill from DeepPavlov. We will train this skill with our dialogue data. 

Skills in DeepPavlov are defined by configuration files. So, we will use [minimal DSTC2 bot config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot/gobot_dstc2_minimal.json) ([more configs](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/go_bot) are available) and change sections responsible for 
- embeddings,
- response templates,
- data and model load/save paths.

Loading bot:

In [ ]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2_minimal)

Download pre-trained GLOVe embeddings:

In [ ]:
from deeppavlov.download import download_resource

download_resource(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
                  dest_paths=['assistant_bot/'])

Configure bot to use downloaded embeddings:

In [ ]:
gobot_config['chainer']['pipe'][-1]['embedder'] = {
    "class_name": "glove",
    "load_path": "assistant_bot/glove.6B.100d.txt"
}

Configure bot to use templates:

In [ ]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'assistant_data/assistant-templates.txt'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['api_call_action'] = None

Specify train/valid/test data path and path to save the final bot model:

In [ ]:
gobot_config['dataset_reader']['class_name'] = '__main__:AssistantDatasetReader'
gobot_config['metadata']['variables']['DATA_PATH'] = 'assistant_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'assistant_bot'

The whole dialogue system pipeline looks like this:
    
![gobot_simple_pipeline.png](img/gobot_simple_pipeline.png)

Train policy network:

In [ ]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 4 # set batch size
gobot_config['train']['max_batches'] = 30 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 30 # evaluate on full 'valid' split every 30 epochs
gobot_config['train']['log_every_n_batches'] = 5 # evaluate on full 'train' split every 5 batches

train_model(gobot_config);

Training on the dataset takes up to 5 minutes depending on gpu/cpu. See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of the training process.

# 2. Interact with bot

In [ ]:
from deeppavlov import build_model

bot = build_model(gobot_config)

In [ ]:
bot([[{"text": "good evening, bot"}]])

In [ ]:
bot([[{"text": "the weather is clooudy and gloooomy"}]])

In [ ]:
bot([[{"text": "nice idea, thanks!"}]])

In [ ]:
bot.reset()

In [ ]:
bot([[{"text": "hi bot"}]])

In [ ]:
bot([[{"text": "looks ok, the sun is bright and yesterday's rain stopped already"}]])

In [ ]:
bot([[{"text": "i dont wanna"}]])

You can also train a more advanced goal-oriented bot following [gobot_extended_tutorial.ipynb](https://github.com/deepmipt/DeepPavlov/blob/master/examples/gobot_extended_tutorial.ipynb)